## Projeto 2 Ciência dos Dados 2020.1
___
- Daniel Gurgel Terra
- Gabriel Formario
___

In [1]:
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import *
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Data

In [3]:
data = pd.read_csv("mushrooms.csv")

## Funções

* Json

In [4]:
import json
with open('Renomeador.json', encoding="utf-8") as json_file: 
    renomeador = json.load(json_file)

* Limpeza dos dados 

In [5]:
def Limpeza(data):
    i = 0
    for nomeClasse,listaValores in renomeador.items():
        data.rename(columns={data.columns[i]:nomeClasse}, inplace=True)
        data[nomeClasse].replace(listaValores, inplace=True)
        i += 1

* Gráfico para cada característica do cogumelo no dataset

In [6]:
# Dataset passa pela limpeza para que a função abaixo funcione
Limpeza(data)

In [7]:
def Grafico_por_caracteristica(intervalo,data):

    colunas = list(renomeador.keys())

    for i in intervalo:

        plt.figure(figsize=(10,5))

        Venenoso = data[data['Periculosidade']=='venenoso'][colunas[i]].value_counts(True)*100
        Comestivel = data[data['Periculosidade']=='comestível'][colunas[i]].value_counts(True)*100

        plt.bar( np.arange(len(Comestivel)), Comestivel.values.tolist(), width=.25, color="g")

        plt.bar( np.arange(len(Venenoso)) + [.25]*len(Venenoso), Venenoso.values.tolist(), width=.25, color="r")

        plt.xticks(np.arange(len(Venenoso)) + len(Venenoso)*[0.125],Venenoso.axes[0].tolist())

        plt.ylabel("Percentual"); plt.xlabel("Características"); plt.title(colunas[i])
        
        plt.legend(['Comestível','Venenoso'],title="Periculosidade")

    plt.show()

 #### Encoder
 * Processo que transforma as variavéis qualitativas em quantitativas de acordo com a respectiva característica
 * Técnica encontrada em : https://www.kaggle.com/nirajvermafcb/comparing-various-ml-models-roc-curve-comparison

In [8]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
for col in data.columns:
    data[col] = labelencoder.fit_transform(data[col])

# Mostrar que o dataset agora possui uma numeração para cada característica   
data['Ligação Branquea'].unique()

array([1, 0])

In [9]:
inputs = data.drop(columns=["Periculosidade"])
target = data["Periculosidade"]
X_treino,X_teste,Y_treino,Y_teste=tts(inputs,target,test_size=0.8)

* Funcao de previsao de modelos utilizadas no Machine Learning

In [10]:
def Previsão(tipo,logistica=False):
    
    if logistica:
        model = tipo(solver='lbfgs',multi_class='auto',max_iter=1000)
    else:
        model = tipo()
    
    model.fit(X_treino,Y_treino);
    model.score(X_teste,Y_teste);
    Y_predict = model.predict(X_teste);
    
    acertos=0
    for i in range(len(X_teste)):
        if Y_predict[i]==Y_teste.iloc[i]:
            acertos+=1;
            
    erros = len(X_teste)-acertos
    
    return round(acertos/len(X_teste)*100,2), round(erros/len(X_teste)*100,2)